In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import plotly.express as px # to create interactive visualizations
from plotly.offline import init_notebook_mode, iplot
import os
import matplotlib.pyplot as plt
import seaborn as sns
init_notebook_mode(connected = True)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install chart-studio

In [ ]:
import chart_studio
username='AbhishekVIzPlotly'
api_key='rCrdhzpUi4b0VvJCVa2V'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)

In [ ]:
import chart_studio.plotly as py
import chart_studio.tools as tls

## World Happiness Dataset 2021 
  

## Definitions of measured parameters
**Ladder score** : The rankings of national happiness are based on a **Cantril ladder survey**. Nationally representative samples of respondents are asked to think of a ladder, with the best possible life for them being a 10, and the worst possible life being a 0. They are then asked to rate their own current lives on that 0 to 10 scale.

**Logged GDP per capita** : Log10 value of the GDP per Capita of the country.This is arguably the most accepted method of determining a country’s level of wealth. And as most of you know, a country’s wealth is highly correlated to its happiness

**Social support** : Social support schemes being implemented by Government and other organizations.If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?” People were only able to answer with a yes or no. A yes would equal 1 and a no would equal 0. The average of all answers results in a single value that represents the amount of social support that is present in a country.

**Healthy life expectancy** : The healthy life expectancy measure adds a 'quality of life' dimension to estimates of life expectancy by dividing it into time spent in different states of health. The number of years of life in poor health is also important as it relates more closely to the demand for health and social care and the associated costs

**Freedom to make life choices** : “Freedom to make life choices” is the national average of responses to the question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

**Generosity** : Have you donated money to a charity in the past month?”
Again, the average of all responses (yes being 1 and no being 0) determines the output of this key factor.It can also be a charitable work from an individual in other terms too.

**Perceptions of corruption** : “Perceptions of corruption” is also determined as a key factor in the Happiness Index. It is calculated by averaging the answers to the following 2 questions:
“Is corruption widespread throughout the government or not?”
“Is corruption widespread within businesses or not?”
 Again, the average of all answers determines the outcome of this factor per country 
 
 **Dystopia+Residual**: Dystopia happiness score is the score of a hypothetical country that has a lower rank than the lowest ranking country on the report, plus the residual value of each country (a number that is left over from the normalization of the variables which cannot be explained).**Residual is the unexplained happiness which cannot be measured like that!!**
 
 Credits: trackinghappiness.com

In [ ]:
data=pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv')
data.head()

 # - Visualization Project - using Seaborn and Plotly

#### Ladder score is the total score used for ranking

In [ ]:
all=data['Country name']
df=pd.DataFrame([data['Ladder score'],data['Country name']]).T.sort_values(by='Ladder score')
df.head()

In [ ]:
data.info()

### Let's visualize the countries in order of their happiness index

In [ ]:
sns.set(rc={'figure.figsize':(16,40)})
sns.set_style("darkgrid")
plt.style.use("dark_background")
plt.title("Happiness index vs Countries")
fig=sns.barplot(data=df.sort_values(by=['Ladder score'],ascending=False),x='Ladder score',y='Country name');

## Ladder score vs Countries with Plotly interactive viz

In [ ]:
i=10
for parameter in data.columns[6:12]:
    figg=px.bar(df,x='Ladder score',y='Country name',color=data[parameter],orientation='h',height=2500,title=f'Ladder score with "{parameter}" coloring ') 
    figg.update_layout(template='plotly_dark',autosize=False,width=600,height=2500)
    figg.show()
    py.plot(figg, filename=f"plotly{i}", auto_open = True)
    i+=1
   

### Visualization of correlation using Heatmap

In [ ]:
data.drop(columns='Ladder score in Dystopia',inplace=True)
corrvalues=[]
for feature in data.columns[6:]:
    laddercorr=data['Ladder score'].corr(data[feature])
    corrvalues.append(laddercorr)
corrvalues.sort(reverse=True)
corrvalues

In [ ]:

import seaborn as sns
import plotly.express as px
sns.set(rc={'figure.figsize':(12,8)})
fig=sns.heatmap(data.iloc[:,2:].corr(),annot=False)


## Correlation bar chart for parameters used in measuring Happiness index

In [ ]:
fig=px.bar(x=corrvalues,y=data.columns[6:],labels={'x':'Ladder score(Total score used for ranking)'},barmode='group')
fig.update_layout(template='plotly_dark',title='Correlation between Ladder score and parameters measured for happiness index',autosize=True)
py.plot(fig, filename="plotly1", auto_open = True)

### Grouping by region where the countries are located with their average Ladder score

In [ ]:
region=data.groupby(['Regional indicator']).mean().drop(columns=['Standard error of ladder score','upperwhisker','lowerwhisker','Explained by: Log GDP per capita','Explained by: Social support','Explained by: Healthy life expectancy','Explained by: Freedom to make life choices','Explained by: Generosity','Explained by: Perceptions of corruption'])
region=pd.DataFrame(region)
region.reset_index()
regions=list(region.index.values)
region

### Country region and their Happiness index using averaged Ladder scores of that region

In [ ]:
fig_region=px.bar(region,y=regions,x='Ladder score',color=region['Ladder score'],orientation='h',title='Region vs Total Happiness index').update_yaxes(categoryorder='total ascending')
fig_region.update_layout(template='plotly_dark',autosize=True)
fig_region.show()
py.plot(fig_region, filename="plotly2", auto_open = True)

#### Lower number of countries gave a higher ladder score to North America and ANZ

In [ ]:
data[data['Regional indicator']=='North America and ANZ']

### Pycountry is imported to render a world map using ISO Alpha 3 key codes for countries

In [ ]:
import pycountry
input_countries = list(data['Country name'])
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

codes = [countries.get(country, 'Unknown code') for country in input_countries]

#print(codes)  
data['Country code']=codes


### Choropleth map to visualize happiness index in a world map

In [ ]:
chorofig=px.choropleth(data.drop(columns=['Regional indicator']),locations=data['Country code'],color='Ladder score',projection='natural earth',color_continuous_scale='Viridis',title='Happiness index of countries in world map(higher ladder score means happier)')
chorofig.show()
py.plot(chorofig, filename="plotly3", auto_open = True)

### Highly interactive Sunburst Chart from Plotly with Social support gradient
##### Hover and click to expand the chart and see the details

In [ ]:
sns.set()
sunburst=px.sunburst(data,path=['Regional indicator','Country name'],values=data['Ladder score'],color='Social support',title='Happiness indices colored by social support',width=600,height=600)
sunburst.show()
py.plot(sunburst, filename="plotly4", auto_open = True)

In [ ]:
sunburst=px.sunburst(data,path=['Regional indicator','Country name'],values=data['Ladder score'],color='Perceptions of corruption',title='Happiness indices colored by Perceptions of corruption',width=600,height=600)
sunburst.show()
py.plot(sunburst, filename="plotly5", auto_open = True)

## Let's see the amount of happiness explained by these factors for each country
#### Sum of all "Explained" values = total Ladder score

In [ ]:
plotsum = data[['Country name','Ladder score','Explained by: Log GDP per capita', 'Explained by: Social support', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption', 'Dystopia + residual']].sort_values(by=['Ladder score'])
plotsum.set_index('Country name').drop(columns=['Ladder score']).plot(kind='barh',xticks=np.arange(10), stacked=True, figsize= (10,80))
plt.xlabel("Happiness Score")
plt.title('Happiness Score of Countries')
plt.style.use(['dark_background'])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()


### __ Abhishek __